In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
def QnA(
    art,
    client,
    model,
    QnA_prompt,
):

    prompt_complete = QnA_prompt + "\n" + art["title"] + "\n" + art["body"]
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [3]:
import json


def get_LLM_QnA(
    src_filename,
    client,
    model="gpt-4o",
    QnA_prompt="",
):
    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)
        for src_art in src_articles:
            src_art["summary"] = QnA(
                src_art,
                client,
                model=model,
                QnA_prompt=QnA_prompt,
            )

            des_articles.append(src_art)
    des_filename = "summary_" + src_filename
    if os.path.exists(des_filename):
        os.remove(des_filename)
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [4]:
question = "Please summarize the following news article in English. The summary should be concise, and should include the main points, key events, people involved, and any significant outcomes or implications:"
get_LLM_QnA(
    "50_combined_Territorial_disputes_in_the_South_China_Sea.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
)